# 📋 GENERADOR AUTOMÁTICO DE ÓRDENES DE TRABAJO
## Sistema de Mantenimiento Predictivo con IA

**Autor:** Antonio Cantos & Renzo Chavez - TFM EADIC - 2025

### 🎯 OBJETIVO:
Generar automáticamente órdenes de trabajo (OTs) basadas en anomalías detectadas por el sistema de IA, con prescripciones específicas, prioridades y recursos necesarios.

### 🔧 FUNCIONALIDADES:
1. **Carga de anomalías detectadas** → Desde sistema de detección
2. **Clasificación automática** → Por severidad y tipo de intervención
3. **Generación de OTs** → Con prescripciones específicas
4. **Asignación de recursos** → Especialidades y herramientas
5. **Programación temporal** → Fechas y tiempos de respuesta
6. **Sistema de alertas** → Notificaciones automáticas
7. **Exportación** → Formatos estándar (CSV, Excel)

### ⚡ VENTAJAS:
- **Generación automática** sin intervención humana
- **Prescripciones específicas** basadas en tipo de anomalía
- **Priorización inteligente** según severidad
- **Asignación de recursos** optimizada
- **Trazabilidad completa** desde detección hasta ejecución

In [ ]:
# 1. 🔧 CONFIGURACIÓN E IMPORTACIONES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import os
from datetime import datetime, timedelta
import json
import uuid
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('default')

# Rutas de trabajo
BASE_PATH = Path('.')
OUTPUT_PATH = BASE_PATH / 'output'
OTS_PATH = OUTPUT_PATH / 'ots_generadas'

# Crear directorios
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OTS_PATH.mkdir(parents=True, exist_ok=True)

print("📋 GENERADOR AUTOMÁTICO DE OTs INICIADO")
print(f"📁 Directorio base: {BASE_PATH}")
print(f"📁 Salida OTs: {OTS_PATH}")
print("=" * 50)

In [ ]:
# 2. 📥 CARGAR ANOMALÍAS DETECTADAS
print("📥 CARGANDO ANOMALÍAS DETECTADAS")
print("=" * 35)

# Buscar archivo de anomalías detectadas
posibles_archivos_anomalias = [
    OUTPUT_PATH / 'anomalias_detectadas_agosto.csv',
    OUTPUT_PATH / 'resultados_deteccion_completos.csv',
    BASE_PATH / 'anomalias_detectadas_agosto.csv'
]

anomalias_detectadas = None
archivo_anomalias_encontrado = None

for archivo in posibles_archivos_anomalias:
    if archivo.exists():
        try:
            print(f"🔄 Intentando cargar: {archivo}")
            df_temp = pd.read_csv(archivo)
            
            # Verificar si tiene columnas de anomalías
            if 'es_anomalia' in df_temp.columns:
                # Filtrar solo anomalías
                anomalias_detectadas = df_temp[df_temp['es_anomalia'] == True].copy()
            elif 'severidad' in df_temp.columns:
                # Ya es un archivo de anomalías
                anomalias_detectadas = df_temp.copy()
            else:
                continue
            
            archivo_anomalias_encontrado = archivo
            print(f"✅ Anomalías cargadas desde: {archivo}")
            break
            
        except Exception as e:
            print(f"❌ Error cargando {archivo}: {e}")
            continue

# Si no se encontraron anomalías, crear datos de ejemplo
if anomalias_detectadas is None or len(anomalias_detectadas) == 0:
    print("\n⚠️ NO SE ENCONTRARON ANOMALÍAS DETECTADAS")
    print("🔧 CREANDO DATOS DE EJEMPLO PARA DEMOSTRACIÓN...")
    
    # Crear anomalías de ejemplo
    fechas_ejemplo = pd.date_range(start='2025-08-01', end='2025-08-31', freq='6H')
    np.random.seed(42)
    
    # Seleccionar algunas fechas aleatorias para anomalías
    indices_anomalias = np.random.choice(len(fechas_ejemplo), size=15, replace=False)
    fechas_anomalias = fechas_ejemplo[indices_anomalias]
    
    anomalias_ejemplo = []
    severidades = ['CRÍTICO', 'ALERTA', 'ATENCIÓN', 'MENOR']
    pesos_severidad = [0.2, 0.3, 0.3, 0.2]  # Distribución de severidades
    
    for i, fecha in enumerate(fechas_anomalias):
        severidad = np.random.choice(severidades, p=pesos_severidad)
        
        # THD basado en severidad
        if severidad == 'CRÍTICO':
            thd = np.random.uniform(4.0, 6.0)
            score = np.random.uniform(-0.8, -0.5)
        elif severidad == 'ALERTA':
            thd = np.random.uniform(2.0, 4.0)
            score = np.random.uniform(-0.5, -0.3)
        elif severidad == 'ATENCIÓN':
            thd = np.random.uniform(1.5, 2.0)
            score = np.random.uniform(-0.3, -0.1)
        else:  # MENOR
            thd = np.random.uniform(1.0, 1.5)
            score = np.random.uniform(-0.2, -0.05)
        
        anomalia = {
            'timestamp': fecha,
            'THD': thd,
            'score_anomalia': score,
            'severidad': severidad,
            'diagnostico': f"Anomalía {severidad.lower()} detectada - THD: {thd:.3f} - Score: {score:.3f}",
            'es_anomalia': True,
            'Estado': 'FUNCIONANDO'
        }
        anomalias_ejemplo.append(anomalia)
    
    anomalias_detectadas = pd.DataFrame(anomalias_ejemplo)
    print(f"✅ Creadas {len(anomalias_detectadas)} anomalías de ejemplo")

# Procesar timestamp si es string
if 'timestamp' in anomalias_detectadas.columns:
    if anomalias_detectadas['timestamp'].dtype == 'object':
        anomalias_detectadas['timestamp'] = pd.to_datetime(anomalias_detectadas['timestamp'])

# Mostrar resumen de anomalías
print(f"\n📊 RESUMEN DE ANOMALÍAS CARGADAS:")
print(f"   Total anomalías: {len(anomalias_detectadas)}")

if len(anomalias_detectadas) > 0:
    if 'timestamp' in anomalias_detectadas.columns:
        print(f"   Período: {anomalias_detectadas['timestamp'].min()} a {anomalias_detectadas['timestamp'].max()}")
    
    if 'severidad' in anomalias_detectadas.columns:
        severidad_counts = anomalias_detectadas['severidad'].value_counts()
        print(f"\n   Distribución por severidad:")
        for severidad, cantidad in severidad_counts.items():
            print(f"     {severidad}: {cantidad} anomalías")
    
    print(f"\n   Columnas disponibles: {list(anomalias_detectadas.columns)}")

print("\n✅ ANOMALÍAS CARGADAS - LISTAS PARA GENERAR OTs")

In [ ]:
# 3. 🏭 CONFIGURACIÓN DEL SISTEMA DE OTs
print("🏭 CONFIGURANDO SISTEMA DE GENERACIÓN DE OTs")
print("=" * 45)

# Configuración de la planta
config_planta = {
    'nombre_planta': 'Frío Pacífico 1',
    'ubicacion': 'Sala de Compresores - Planta Refrigeración',
    'equipo_principal': 'Compresor C1',
    'responsable_mantenimiento': 'Jefe de Mantenimiento',
    'turno_operacion': '24/7',
    'codigo_planta': 'FP1'
}

# Mapeo de severidad a tipo de OT
mapeo_tipo_ot = {
    'CRÍTICO': 'CORRECTIVO',
    'ALERTA': 'PREDICTIVO', 
    'ATENCIÓN': 'PREVENTIVO',
    'MENOR': 'INSPECCIÓN'
}

# Mapeo de severidad a prioridad (1=más alta, 4=más baja)
mapeo_prioridad = {
    'CRÍTICO': 1,
    'ALERTA': 2,
    'ATENCIÓN': 3,
    'MENOR': 4
}

# Mapeo de severidad a tiempo de respuesta
mapeo_tiempo_respuesta = {
    'CRÍTICO': {'horas': 2, 'descripcion': '2 horas'},
    'ALERTA': {'horas': 8, 'descripcion': '8 horas'},
    'ATENCIÓN': {'horas': 24, 'descripcion': '24 horas'},
    'MENOR': {'horas': 168, 'descripcion': '1 semana'}  # 7 días
}

# Especialidades requeridas por tipo de problema
especialidades = {
    'electrica': {
        'nombre': 'Eléctrica',
        'descripcion': 'Especialista en sistemas eléctricos industriales',
        'criterio': 'THD > 1.5 o problemas eléctricos'
    },
    'mecanica': {
        'nombre': 'Mecánica',
        'descripcion': 'Especialista en sistemas mecánicos y refrigeración',
        'criterio': 'Problemas mecánicos o térmicos'
    },
    'instrumentacion': {
        'nombre': 'Instrumentación',
        'descripcion': 'Especialista en sensores y sistemas de control',
        'criterio': 'Problemas de medición o control'
    }
}

# Recursos y herramientas por especialidad
recursos_por_especialidad = {
    'electrica': [
        'Analizador de red trifásico',
        'Multímetro digital de precisión',
        'Osciloscopio',
        'Medidor de aislamiento',
        'Herramientas eléctricas básicas',
        'Equipo de protección personal eléctrico'
    ],
    'mecanica': [
        'Analizador de vibraciones',
        'Termómetro infrarrojo',
        'Manómetros de precisión',
        'Herramientas mecánicas básicas',
        'Lubricantes y refrigerantes',
        'Equipo de protección personal'
    ],
    'instrumentacion': [
        'Calibrador de procesos',
        'Multímetro de procesos',
        'Generador de señales',
        'Herramientas de instrumentación',
        'Cables y conectores de repuesto'
    ]
}

# Contactos de emergencia por severidad
contactos_emergencia = {
    'CRÍTICO': [
        'jefe.mantenimiento@friop1.com',
        'supervisor.turno@friop1.com',
        'gerente.operaciones@friop1.com'
    ],
    'ALERTA': [
        'jefe.mantenimiento@friop1.com',
        'supervisor.turno@friop1.com'
    ],
    'ATENCIÓN': [
        'planificador.mantenimiento@friop1.com'
    ],
    'MENOR': [
        'planificador.mantenimiento@friop1.com'
    ]
}

print(f"✅ Configuración del sistema:")
print(f"   Planta: {config_planta['nombre_planta']}")
print(f"   Equipo: {config_planta['equipo_principal']}")
print(f"   Tipos de OT: {list(mapeo_tipo_ot.values())}")
print(f"   Especialidades: {[esp['nombre'] for esp in especialidades.values()]}")
print(f"   Niveles de prioridad: 1 (crítica) a 4 (menor)")

print("\n✅ SISTEMA DE OTs CONFIGURADO")

In [ ]:
# 4. 🔧 GENERADOR DE PRESCRIPCIONES ESPECÍFICAS
print("🔧 CONFIGURANDO GENERADOR DE PRESCRIPCIONES")
print("=" * 45)

def generar_prescripciones_especificas(severidad, thd, score_anomalia, timestamp):
    """Genera prescripciones específicas basadas en la anomalía detectada"""
    
    # Prescripciones base por severidad
    prescripciones_base = {
        'CRÍTICO': {
            'inmediatas': [
                "🚨 PARADA INMEDIATA del compresor para evaluación de seguridad",
                "⚡ Verificar conexiones eléctricas principales y tablero de control",
                "📊 Medir THD en tiempo real con analizador de red trifásico",
                "🔧 Revisar estado de contactores, relés y protecciones eléctricas"
            ],
            'seguimiento': [
                "🔍 Evaluar calidad de red eléctrica y presencia de armónicos",
                "📦 Preparar repuestos críticos (contactores, relés, fusibles)",
                "👨‍🔧 Coordinar con electricista especializado en sistemas industriales",
                "📋 Documentar todas las mediciones y hallazgos"
            ],
            'preventivas': [
                "🛡️ Evaluar instalación de filtros armónicos",
                "📈 Implementar monitoreo continuo de THD",
                "🔄 Revisar programa de mantenimiento preventivo eléctrico"
            ]
        },
        'ALERTA': {
            'inmediatas': [
                "⏰ Programar inspección eléctrica especializada en próximas 8 horas",
                "📊 Monitorear THD cada 2 horas durante las próximas 24 horas",
                "🔧 Verificar apriete de conexiones eléctricas principales"
            ],
            'seguimiento': [
                "🔍 Revisar funcionamiento de filtros armónicos si existen",
                "📋 Documentar evolución de parámetros eléctricos",
                "⚡ Evaluar necesidad de instalación de filtros armónicos",
                "📞 Notificar a supervisor de turno sobre anomalía"
            ],
            'preventivas': [
                "📅 Incluir en próximo mantenimiento preventivo",
                "🔄 Revisar frecuencia de inspecciones eléctricas"
            ]
        },
        'ATENCIÓN': {
            'inmediatas': [
                "📅 Incluir en próximo mantenimiento preventivo programado",
                "📊 Monitorear THD diariamente durante una semana"
            ],
            'seguimiento': [
                "🌡️ Verificar carga térmica del sistema de refrigeración",
                "🧹 Revisar limpieza de intercambiadores de calor",
                "⚡ Evaluar patrones de operación y cargas eléctricas",
                "📋 Documentar tendencias de parámetros"
            ],
            'preventivas': [
                "🔄 Ajustar frecuencia de monitoreo si es necesario",
                "📈 Incluir en análisis de tendencias mensuales"
            ]
        },
        'MENOR': {
            'inmediatas': [
                "📊 Incluir en monitoreo rutinario semanal",
                "📋 Registrar en inspección mensual de parámetros eléctricos"
            ],
            'seguimiento': [
                "📈 Verificar tendencias a largo plazo en próxima revisión",
                "📊 Documentar para análisis de patrones estacionales",
                "🔍 Incluir en reporte mensual de condición"
            ],
            'preventivas': [
                "📅 Mantener en programa de monitoreo regular"
            ]
        }
    }
    
    # Prescripciones específicas basadas en THD
    prescripciones_thd = []
    
    if thd > 5.0:
        prescripciones_thd.extend([
            f"🚨 THD EXTREMO ({thd:.3f}) - Riesgo de daño a equipos",
            "⚡ Verificar inmediatamente calidad de suministro eléctrico",
            "🔧 Revisar cargas no lineales conectadas al sistema"
        ])
    elif thd > 3.0:
        prescripciones_thd.extend([
            f"⚠️ THD ELEVADO ({thd:.3f}) - Monitoreo intensivo requerido",
            "📊 Realizar análisis espectral de armónicos",
            "🔍 Identificar fuentes de distorsión armónica"
        ])
    elif thd > 1.5:
        prescripciones_thd.extend([
            f"📈 THD MODERADO ({thd:.3f}) - Seguimiento requerido",
            "📊 Incluir en monitoreo regular de calidad eléctrica"
        ])
    
    # Prescripciones basadas en score de anomalía
    prescripciones_score = []
    
    if score_anomalia < -0.6:
        prescripciones_score.append("🎯 ANOMALÍA MUY SIGNIFICATIVA - Investigación profunda requerida")
    elif score_anomalia < -0.3:
        prescripciones_score.append("🔍 ANOMALÍA SIGNIFICATIVA - Análisis detallado recomendado")
    elif score_anomalia < -0.1:
        prescripciones_score.append("📊 ANOMALÍA MODERADA - Monitoreo continuo")
    
    # Prescripciones basadas en horario
    prescripciones_horario = []
    hora = timestamp.hour if hasattr(timestamp, 'hour') else 12
    
    if 22 <= hora or hora <= 6:  # Turno nocturno
        prescripciones_horario.extend([
            "🌙 TURNO NOCTURNO - Coordinar con supervisor de turno",
            "📞 Notificar a personal de guardia si es crítico"
        ])
    elif 6 < hora < 14:  # Turno mañana
        prescripciones_horario.append("🌅 TURNO MAÑANA - Coordinar con equipo diurno")
    elif 14 <= hora < 22:  # Turno tarde
        prescripciones_horario.append("🌆 TURNO TARDE - Verificar disponibilidad de recursos")
    
    # Combinar todas las prescripciones
    prescripciones_completas = []
    
    # Agregar prescripciones base
    if severidad in prescripciones_base:
        prescripciones_completas.extend(prescripciones_base[severidad]['inmediatas'])
        prescripciones_completas.extend(prescripciones_base[severidad]['seguimiento'])
        prescripciones_completas.extend(prescripciones_base[severidad]['preventivas'])
    
    # Agregar prescripciones específicas
    prescripciones_completas.extend(prescripciones_thd)
    prescripciones_completas.extend(prescripciones_score)
    prescripciones_completas.extend(prescripciones_horario)
    
    return prescripciones_completas

def determinar_especialidad(thd, severidad, score_anomalia):
    """Determina la especialidad requerida basada en los parámetros"""
    
    if thd > 1.5 or severidad in ['CRÍTICO', 'ALERTA']:
        return 'electrica'
    elif score_anomalia < -0.3:
        return 'instrumentacion'
    else:
        return 'mecanica'

def calcular_duracion_estimada(severidad, thd):
    """Calcula la duración estimada de la intervención"""
    
    duraciones_base = {
        'CRÍTICO': {'min': 4, 'max': 12},  # horas
        'ALERTA': {'min': 2, 'max': 6},
        'ATENCIÓN': {'min': 1, 'max': 3},
        'MENOR': {'min': 0.5, 'max': 1}
    }
    
    base = duraciones_base.get(severidad, {'min': 1, 'max': 2})
    
    # Ajustar por THD
    if thd > 4.0:
        multiplicador = 1.5
    elif thd > 2.0:
        multiplicador = 1.2
    else:
        multiplicador = 1.0
    
    duracion_min = base['min'] * multiplicador
    duracion_max = base['max'] * multiplicador
    
    return {
        'minima': duracion_min,
        'maxima': duracion_max,
        'estimada': (duracion_min + duracion_max) / 2
    }

print("✅ Generador de prescripciones configurado")
print("   - Prescripciones específicas por severidad")
print("   - Análisis de THD y score de anomalía")
print("   - Consideraciones de turno y horario")
print("   - Determinación automática de especialidad")
print("   - Cálculo de duración estimada")

In [ ]:
# 5. 📋 GENERACIÓN AUTOMÁTICA DE OTs
print("📋 GENERANDO ÓRDENES DE TRABAJO AUTOMÁTICAMENTE")
print("=" * 50)

def generar_ot_completa(anomalia_row, numero_secuencial):
    """Genera una OT completa basada en la anomalía detectada"""
    
    # Extraer datos de la anomalía
    timestamp = anomalia_row['timestamp']
    severidad = anomalia_row.get('severidad', 'MENOR')
    thd = anomalia_row.get('THD', 0)
    score = anomalia_row.get('score_anomalia', 0)
    diagnostico_ia = anomalia_row.get('diagnostico', 'Anomalía detectada por IA')
    
    # Generar ID único para la OT
    fecha_str = timestamp.strftime('%Y%m%d')
    hora_str = timestamp.strftime('%H%M')
    numero_ot = f"IA-{config_planta['codigo_planta']}-{numero_secuencial:04d}-{fecha_str}-{hora_str}"
    
    # Determinar tipo de OT y prioridad
    tipo_ot = mapeo_tipo_ot[severidad]
    prioridad = mapeo_prioridad[severidad]
    tiempo_respuesta = mapeo_tiempo_respuesta[severidad]
    
    # Calcular fechas
    fecha_deteccion = timestamp
    fecha_programada = timestamp + timedelta(hours=tiempo_respuesta['horas'])
    fecha_limite = fecha_programada + timedelta(hours=tiempo_respuesta['horas'])
    
    # Determinar especialidad y recursos
    especialidad_key = determinar_especialidad(thd, severidad, score)
    especialidad = especialidades[especialidad_key]
    recursos_necesarios = recursos_por_especialidad[especialidad_key]
    
    # Generar prescripciones específicas
    prescripciones = generar_prescripciones_especificas(severidad, thd, score, timestamp)
    
    # Calcular duración estimada
    duracion = calcular_duracion_estimada(severidad, thd)
    
    # Determinar contactos
    contactos = contactos_emergencia[severidad]
    
    # Crear OT completa
    ot = {
        # Identificación
        'numero_ot': numero_ot,
        'id_unico': str(uuid.uuid4()),
        'tipo': tipo_ot,
        'prioridad': prioridad,
        'severidad': severidad,
        
        # Fechas y tiempos
        'fecha_deteccion': fecha_deteccion,
        'fecha_generacion': datetime.now(),
        'fecha_programada': fecha_programada,
        'fecha_limite': fecha_limite,
        'tiempo_respuesta': tiempo_respuesta['descripcion'],
        'duracion_estimada_horas': duracion['estimada'],
        'duracion_minima_horas': duracion['minima'],
        'duracion_maxima_horas': duracion['maxima'],
        
        # Equipo y ubicación
        'equipo': config_planta['equipo_principal'],
        'ubicacion': config_planta['ubicacion'],
        'planta': config_planta['nombre_planta'],
        'codigo_planta': config_planta['codigo_planta'],
        
        # Descripción y diagnóstico
        'titulo': f"Anomalía {severidad} - THD {thd:.3f} - {tipo_ot}",
        'descripcion': f"Anomalía {severidad.lower()} detectada automáticamente por sistema IA. THD: {thd:.3f}, Score: {score:.3f}. Requiere intervención {tipo_ot.lower()}.",
        'diagnostico_ia': diagnostico_ia,
        'sintomas': f"THD elevado ({thd:.3f}), Score de anomalía: {score:.3f}, Patrón anómalo detectado en {timestamp.strftime('%Y-%m-%d %H:%M')}",
        
        # Parámetros técnicos
        'thd_detectado': thd,
        'score_anomalia': score,
        'metodo_deteccion': 'Isolation Forest + Análisis THD',
        'confianza_deteccion': min(abs(score) * 100, 100),  # Convertir a porcentaje
        
        # Recursos y personal
        'especialidad_requerida': especialidad['nombre'],
        'descripcion_especialidad': especialidad['descripcion'],
        'recursos_necesarios': recursos_necesarios,
        'personal_estimado': 2 if severidad in ['CRÍTICO', 'ALERTA'] else 1,
        
        # Prescripciones y procedimientos
        'prescripciones': prescripciones,
        'numero_prescripciones': len(prescripciones),
        
        # Contactos y notificaciones
        'contactos_notificar': contactos,
        'responsable_asignado': config_planta['responsable_mantenimiento'],
        
        # Estado y seguimiento
        'estado_ot': 'GENERADA',
        'generado_por': 'Sistema IA Mantenimiento Predictivo',
        'requiere_aprobacion': severidad in ['CRÍTICO', 'ALERTA'],
        'requiere_parada': severidad == 'CRÍTICO',
        
        # Costos estimados (en USD)
        'costo_estimado_mano_obra': duracion['estimada'] * 50 * (2 if severidad in ['CRÍTICO', 'ALERTA'] else 1),
        'costo_estimado_materiales': 200 if severidad == 'CRÍTICO' else 100 if severidad == 'ALERTA' else 50,
        
        # Metadata
        'version_sistema': '1.0',
        'fuente_datos': 'Sensores THD Compresor C1',
        'algoritmo_deteccion': 'Isolation Forest',
        'umbral_thd_critico': 4.0,
        'umbral_thd_alerta': 2.0
    }
    
    # Calcular costo total estimado
    ot['costo_total_estimado'] = ot['costo_estimado_mano_obra'] + ot['costo_estimado_materiales']
    
    return ot

# Generar OTs para todas las anomalías
ots_generadas = []

if len(anomalias_detectadas) > 0:
    print(f"🔧 Generando OTs para {len(anomalias_detectadas)} anomalías...")
    
    # Ordenar anomalías por severidad y fecha
    orden_severidad = {'CRÍTICO': 1, 'ALERTA': 2, 'ATENCIÓN': 3, 'MENOR': 4}
    anomalias_ordenadas = anomalias_detectadas.copy()
    anomalias_ordenadas['orden_severidad'] = anomalias_ordenadas['severidad'].map(orden_severidad)
    anomalias_ordenadas = anomalias_ordenadas.sort_values(['orden_severidad', 'timestamp'])
    
    # Generar OTs
    for i, (idx, anomalia) in enumerate(anomalias_ordenadas.iterrows(), 1):
        ot = generar_ot_completa(anomalia, i)
        ots_generadas.append(ot)
    
    print(f"\n✅ {len(ots_generadas)} ÓRDENES DE TRABAJO GENERADAS")
    
    # Estadísticas de OTs generadas
    ots_df = pd.DataFrame(ots_generadas)
    
    print(f"\n📊 ESTADÍSTICAS DE OTs GENERADAS:")
    
    # Por tipo
    tipos_ot = ots_df['tipo'].value_counts()
    print(f"\n   Por tipo de OT:")
    for tipo, cantidad in tipos_ot.items():
        print(f"     {tipo}: {cantidad} OTs")
    
    # Por severidad
    severidades_ot = ots_df['severidad'].value_counts()
    print(f"\n   Por severidad:")
    for severidad, cantidad in severidades_ot.items():
        print(f"     {severidad}: {cantidad} OTs")
    
    # Por especialidad
    especialidades_ot = ots_df['especialidad_requerida'].value_counts()
    print(f"\n   Por especialidad:")
    for especialidad, cantidad in especialidades_ot.items():
        print(f"     {especialidad}: {cantidad} OTs")
    
    # Costos estimados
    costo_total = ots_df['costo_total_estimado'].sum()
    costo_promedio = ots_df['costo_total_estimado'].mean()
    
    print(f"\n   Costos estimados:")
    print(f"     Total: ${costo_total:,.2f} USD")
    print(f"     Promedio por OT: ${costo_promedio:,.2f} USD")
    
    # Tiempos de respuesta
    duracion_total = ots_df['duracion_estimada_horas'].sum()
    duracion_promedio = ots_df['duracion_estimada_horas'].mean()
    
    print(f"\n   Tiempos estimados:")
    print(f"     Duración total: {duracion_total:.1f} horas")
    print(f"     Duración promedio: {duracion_promedio:.1f} horas por OT")
    
else:
    print("✅ No hay anomalías para generar OTs")

print("\n✅ GENERACIÓN DE OTs COMPLETADA")

In [ ]:
# 6. 📋 MOSTRAR DETALLES DE OTs GENERADAS
print("📋 DETALLES DE ÓRDENES DE TRABAJO GENERADAS")
print("=" * 45)

if len(ots_generadas) > 0:
    # Mostrar las 5 OTs más críticas
    print(f"🚨 TOP 5 OTs MÁS CRÍTICAS:")
    print("=" * 30)
    
    for i, ot in enumerate(ots_generadas[:5], 1):
        print(f"\n🔧 OT #{i}: {ot['numero_ot']}")
        print(f"   📊 Tipo: {ot['tipo']} | Prioridad: {ot['prioridad']} | Severidad: {ot['severidad']}")
        print(f"   📅 Detección: {ot['fecha_deteccion'].strftime('%Y-%m-%d %H:%M')}")
        print(f"   ⏰ Programada: {ot['fecha_programada'].strftime('%Y-%m-%d %H:%M')}")
        print(f"   ⚡ Tiempo respuesta: {ot['tiempo_respuesta']}")
        print(f"   🎯 THD: {ot['thd_detectado']:.3f} | Score: {ot['score_anomalia']:.3f}")
        print(f"   👨‍🔧 Especialidad: {ot['especialidad_requerida']}")
        print(f"   ⏱️ Duración estimada: {ot['duracion_estimada_horas']:.1f} horas")
        print(f"   💰 Costo estimado: ${ot['costo_total_estimado']:.2f} USD")
        print(f"   📋 Título: {ot['titulo']}")
        
        # Mostrar primeras 3 prescripciones
        print(f"   🔧 Prescripciones principales:")
        for j, prescripcion in enumerate(ot['prescripciones'][:3], 1):
            print(f"      {j}. {prescripcion}")
        
        if len(ot['prescripciones']) > 3:
            print(f"      ... y {len(ot['prescripciones'])-3} prescripciones más")
        
        # Mostrar recursos principales
        print(f"   🛠️ Recursos necesarios:")
        for j, recurso in enumerate(ot['recursos_necesarios'][:3], 1):
            print(f"      {j}. {recurso}")
        
        if len(ot['recursos_necesarios']) > 3:
            print(f"      ... y {len(ot['recursos_necesarios'])-3} recursos más")
    
    # Resumen por urgencia
    print(f"\n\n⚡ RESUMEN POR URGENCIA:")
    print("=" * 25)
    
    ots_criticas = [ot for ot in ots_generadas if ot['severidad'] == 'CRÍTICO']
    ots_alertas = [ot for ot in ots_generadas if ot['severidad'] == 'ALERTA']
    ots_atencion = [ot for ot in ots_generadas if ot['severidad'] == 'ATENCIÓN']
    ots_menores = [ot for ot in ots_generadas if ot['severidad'] == 'MENOR']
    
    if ots_criticas:
        print(f"\n🚨 CRÍTICAS ({len(ots_criticas)} OTs):")
        for ot in ots_criticas:
            print(f"   • {ot['numero_ot']} - {ot['fecha_programada'].strftime('%Y-%m-%d %H:%M')} - THD: {ot['thd_detectado']:.3f}")
    
    if ots_alertas:
        print(f"\n🟠 ALERTAS ({len(ots_alertas)} OTs):")
        for ot in ots_alertas:
            print(f"   • {ot['numero_ot']} - {ot['fecha_programada'].strftime('%Y-%m-%d %H:%M')} - THD: {ot['thd_detectado']:.3f}")
    
    if ots_atencion:
        print(f"\n🟡 ATENCIÓN ({len(ots_atencion)} OTs):")
        for ot in ots_atencion[:3]:  # Solo mostrar las primeras 3
            print(f"   • {ot['numero_ot']} - {ot['fecha_programada'].strftime('%Y-%m-%d %H:%M')} - THD: {ot['thd_detectado']:.3f}")
        if len(ots_atencion) > 3:
            print(f"   ... y {len(ots_atencion)-3} más")
    
    if ots_menores:
        print(f"\n🔵 MENORES ({len(ots_menores)} OTs):")
        for ot in ots_menores[:3]:  # Solo mostrar las primeras 3
            print(f"   • {ot['numero_ot']} - {ot['fecha_programada'].strftime('%Y-%m-%d %H:%M')} - THD: {ot['thd_detectado']:.3f}")
        if len(ots_menores) > 3:
            print(f"   ... y {len(ots_menores)-3} más")
    
    # Cronograma de las próximas 48 horas
    print(f"\n\n📅 CRONOGRAMA PRÓXIMAS 48 HORAS:")
    print("=" * 35)
    
    ahora = datetime.now()
    limite_48h = ahora + timedelta(hours=48)
    
    ots_proximas = [ot for ot in ots_generadas if ot['fecha_programada'] <= limite_48h]
    ots_proximas_ordenadas = sorted(ots_proximas, key=lambda x: x['fecha_programada'])
    
    if ots_proximas_ordenadas:
        for ot in ots_proximas_ordenadas:
            fecha_prog = ot['fecha_programada']
            horas_restantes = (fecha_prog - ahora).total_seconds() / 3600
            
            if horas_restantes < 0:
                estado_tiempo = "🚨 VENCIDA"
            elif horas_restantes < 2:
                estado_tiempo = "⚡ URGENTE"
            elif horas_restantes < 8:
                estado_tiempo = "⏰ PRÓXIMA"
            else:
                estado_tiempo = "📅 PROGRAMADA"
            
            print(f"   {estado_tiempo} {fecha_prog.strftime('%m-%d %H:%M')} - {ot['numero_ot']} - {ot['severidad']} - {ot['especialidad_requerida']}")
    else:
        print("   ✅ No hay OTs programadas en las próximas 48 horas")

else:
    print("ℹ️ No hay OTs generadas para mostrar")

print("\n✅ DETALLES DE OTs MOSTRADOS")

In [ ]:
# 7. 💾 EXPORTACIÓN DE OTs EN MÚLTIPLES FORMATOS
print("💾 EXPORTANDO OTs EN MÚLTIPLES FORMATOS")
print("=" * 40)

if len(ots_generadas) > 0:
    
    # 1. Exportar CSV básico
    print("📄 Generando archivo CSV...")
    ots_df = pd.DataFrame(ots_generadas)
    
    # Seleccionar columnas principales para CSV
    columnas_csv = [
        'numero_ot', 'tipo', 'prioridad', 'severidad',
        'fecha_deteccion', 'fecha_programada', 'fecha_limite',
        'equipo', 'ubicacion', 'titulo', 'descripcion',
        'thd_detectado', 'score_anomalia', 'especialidad_requerida',
        'duracion_estimada_horas', 'costo_total_estimado',
        'estado_ot', 'tiempo_respuesta'
    ]
    
    ots_csv = ots_df[columnas_csv].copy()
    archivo_csv = OTS_PATH / f'ots_generadas_{datetime.now().strftime("%Y%m%d_%H%M")}.csv'
    ots_csv.to_csv(archivo_csv, index=False, encoding='utf-8')
    print(f"   ✅ CSV guardado: {archivo_csv}")
    
    # 2. Exportar Excel detallado
    print("📊 Generando archivo Excel detallado...")
    archivo_excel = OTS_PATH / f'ots_detalladas_{datetime.now().strftime("%Y%m%d_%H%M")}.xlsx'
    
    with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
        # Hoja 1: Resumen de OTs
        ots_csv.to_excel(writer, sheet_name='Resumen_OTs', index=False)
        
        # Hoja 2: OTs Críticas
        ots_criticas_df = ots_df[ots_df['severidad'] == 'CRÍTICO']
        if len(ots_criticas_df) > 0:
            ots_criticas_df[columnas_csv].to_excel(writer, sheet_name='OTs_Críticas', index=False)
        
        # Hoja 3: Prescripciones detalladas
        prescripciones_data = []
        for ot in ots_generadas:
            for i, prescripcion in enumerate(ot['prescripciones'], 1):
                prescripciones_data.append({
                    'numero_ot': ot['numero_ot'],
                    'severidad': ot['severidad'],
                    'orden_prescripcion': i,
                    'prescripcion': prescripcion
                })
        
        if prescripciones_data:
            prescripciones_df = pd.DataFrame(prescripciones_data)
            prescripciones_df.to_excel(writer, sheet_name='Prescripciones', index=False)
        
        # Hoja 4: Recursos necesarios
        recursos_data = []
        for ot in ots_generadas:
            for recurso in ot['recursos_necesarios']:
                recursos_data.append({
                    'numero_ot': ot['numero_ot'],
                    'especialidad': ot['especialidad_requerida'],
                    'recurso': recurso
                })
        
        if recursos_data:
            recursos_df = pd.DataFrame(recursos_data)
            recursos_df.to_excel(writer, sheet_name='Recursos', index=False)
        
        # Hoja 5: Estadísticas
        estadisticas_data = {
            'Métrica': [
                'Total OTs Generadas',
                'OTs Críticas',
                'OTs Alerta',
                'OTs Atención',
                'OTs Menores',
                'Costo Total Estimado (USD)',
                'Duración Total Estimada (horas)',
                'Especialidad Eléctrica',
                'Especialidad Mecánica',
                'Especialidad Instrumentación'
            ],
            'Valor': [
                len(ots_generadas),
                len([ot for ot in ots_generadas if ot['severidad'] == 'CRÍTICO']),
                len([ot for ot in ots_generadas if ot['severidad'] == 'ALERTA']),
                len([ot for ot in ots_generadas if ot['severidad'] == 'ATENCIÓN']),
                len([ot for ot in ots_generadas if ot['severidad'] == 'MENOR']),
                f"${sum(ot['costo_total_estimado'] for ot in ots_generadas):,.2f}",
                f"{sum(ot['duracion_estimada_horas'] for ot in ots_generadas):.1f}",
                len([ot for ot in ots_generadas if ot['especialidad_requerida'] == 'Eléctrica']),
                len([ot for ot in ots_generadas if ot['especialidad_requerida'] == 'Mecánica']),
                len([ot for ot in ots_generadas if ot['especialidad_requerida'] == 'Instrumentación'])
            ]
        }
        
        estadisticas_df = pd.DataFrame(estadisticas_data)
        estadisticas_df.to_excel(writer, sheet_name='Estadísticas', index=False)
    
    print(f"   ✅ Excel detallado guardado: {archivo_excel}")
    
    # 3. Exportar JSON para integración con sistemas
    print("🔗 Generando archivo JSON para integración...")
    
    # Preparar datos JSON (convertir datetime a string)
    ots_json = []
    for ot in ots_generadas:
        ot_json = ot.copy()
        # Convertir datetime a string
        for key, value in ot_json.items():
            if isinstance(value, datetime):
                ot_json[key] = value.isoformat()
        ots_json.append(ot_json)
    
    archivo_json = OTS_PATH / f'ots_sistema_{datetime.now().strftime("%Y%m%d_%H%M")}.json'
    with open(archivo_json, 'w', encoding='utf-8') as f:
        json.dump({
            'metadata': {
                'fecha_generacion': datetime.now().isoformat(),
                'total_ots': len(ots_generadas),
                'sistema_generador': 'IA Mantenimiento Predictivo TFM',
                'version': '1.0',
                'planta': config_planta['nombre_planta']
            },
            'ots': ots_json
        }, f, indent=2, ensure_ascii=False)
    
    print(f"   ✅ JSON guardado: {archivo_json}")
    
    # 4. Generar reporte ejecutivo en texto
    print("📋 Generando reporte ejecutivo...")
    
    reporte_ejecutivo = f"""
# 📋 REPORTE EJECUTIVO - ÓRDENES DE TRABAJO GENERADAS
## Sistema de Mantenimiento Predictivo con IA

**Fecha de Generación:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Planta:** {config_planta['nombre_planta']}
**Equipo:** {config_planta['equipo_principal']}
**Sistema Generador:** IA Mantenimiento Predictivo TFM

---

## 📊 RESUMEN EJECUTIVO

### Total de OTs Generadas: {len(ots_generadas)}

**Distribución por Severidad:**
- 🚨 CRÍTICAS: {len([ot for ot in ots_generadas if ot['severidad'] == 'CRÍTICO'])} OTs
- 🟠 ALERTAS: {len([ot for ot in ots_generadas if ot['severidad'] == 'ALERTA'])} OTs
- 🟡 ATENCIÓN: {len([ot for ot in ots_generadas if ot['severidad'] == 'ATENCIÓN'])} OTs
- 🔵 MENORES: {len([ot for ot in ots_generadas if ot['severidad'] == 'MENOR'])} OTs

**Distribución por Tipo:**
"""
    
    tipos_count = {}
    for ot in ots_generadas:
        tipo = ot['tipo']
        tipos_count[tipo] = tipos_count.get(tipo, 0) + 1
    
    for tipo, cantidad in tipos_count.items():
        reporte_ejecutivo += f"- {tipo}: {cantidad} OTs\n"
    
    costo_total = sum(ot['costo_total_estimado'] for ot in ots_generadas)
    duracion_total = sum(ot['duracion_estimada_horas'] for ot in ots_generadas)
    
    reporte_ejecutivo += f"""
**Impacto Económico:**
- Costo Total Estimado: ${costo_total:,.2f} USD
- Costo Promedio por OT: ${costo_total/len(ots_generadas):,.2f} USD
- Duración Total Estimada: {duracion_total:.1f} horas
- Duración Promedio: {duracion_total/len(ots_generadas):.1f} horas por OT

---

## 🚨 ACCIONES INMEDIATAS REQUERIDAS
"""
    
    ots_criticas = [ot for ot in ots_generadas if ot['severidad'] == 'CRÍTICO']
    if ots_criticas:
        reporte_ejecutivo += f"""
### OTs CRÍTICAS ({len(ots_criticas)} OTs):
"""
        for ot in ots_criticas:
            reporte_ejecutivo += f"""
**{ot['numero_ot']}**
- Fecha programada: {ot['fecha_programada'].strftime('%Y-%m-%d %H:%M')}
- THD detectado: {ot['thd_detectado']:.3f}
- Especialidad: {ot['especialidad_requerida']}
- Tiempo respuesta: {ot['tiempo_respuesta']}
- Costo estimado: ${ot['costo_total_estimado']:.2f} USD
"""
    
    reporte_ejecutivo += f"""

---

## 📅 CRONOGRAMA DE EJECUCIÓN

### Próximas 24 Horas:
"""
    
    ahora = datetime.now()
    ots_24h = [ot for ot in ots_generadas if ot['fecha_programada'] <= ahora + timedelta(hours=24)]
    
    if ots_24h:
        for ot in sorted(ots_24h, key=lambda x: x['fecha_programada']):
            reporte_ejecutivo += f"- {ot['fecha_programada'].strftime('%m-%d %H:%M')} - {ot['numero_ot']} - {ot['severidad']}\n"
    else:
        reporte_ejecutivo += "- No hay OTs programadas en las próximas 24 horas\n"
    
    reporte_ejecutivo += f"""

---

## 📁 ARCHIVOS GENERADOS

- `{archivo_csv.name}` - Resumen en CSV
- `{archivo_excel.name}` - Detalles completos en Excel
- `{archivo_json.name}` - Datos para integración JSON
- `reporte_ejecutivo_ots.md` - Este reporte

---

**Generado por:** Sistema IA Mantenimiento Predictivo
**Desarrollado por:** Antonio Cantos & Renzo Chavez - TFM EADIC 2025
**Fecha:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""
    
    archivo_reporte = OTS_PATH / 'reporte_ejecutivo_ots.md'
    with open(archivo_reporte, 'w', encoding='utf-8') as f:
        f.write(reporte_ejecutivo)
    
    print(f"   ✅ Reporte ejecutivo guardado: {archivo_reporte}")
    
    # Resumen de archivos generados
    print(f"\n📁 ARCHIVOS GENERADOS:")
    archivos_generados = list(OTS_PATH.glob('*'))
    for archivo in sorted(archivos_generados, key=lambda x: x.stat().st_mtime, reverse=True):
        if archivo.is_file():
            tamaño = archivo.stat().st_size
            tamaño_mb = tamaño / (1024 * 1024)
            print(f"   📄 {archivo.name} ({tamaño_mb:.2f} MB)")

else:
    print("ℹ️ No hay OTs para exportar")

print("\n✅ EXPORTACIÓN COMPLETADA")

In [ ]:
# 8. 📊 VISUALIZACIÓN DE OTs GENERADAS
print("📊 GENERANDO VISUALIZACIONES DE OTs")
print("=" * 35)

if len(ots_generadas) > 0:
    
    # Configurar estilo
    plt.style.use('default')
    fig_size = (16, 12)
    
    # Crear figura con subplots
    fig, axes = plt.subplots(2, 3, figsize=fig_size)
    fig.suptitle('📋 Análisis de Órdenes de Trabajo Generadas\nSistema IA Mantenimiento Predictivo', 
                 fontsize=16, fontweight='bold')
    
    # Preparar datos
    ots_df = pd.DataFrame(ots_generadas)
    
    # 1. Distribución por severidad
    severidad_counts = ots_df['severidad'].value_counts()
    colores_severidad = {'CRÍTICO': 'red', 'ALERTA': 'orange', 'ATENCIÓN': 'yellow', 'MENOR': 'lightblue'}
    colores = [colores_severidad.get(sev, 'gray') for sev in severidad_counts.index]
    
    bars1 = axes[0,0].bar(severidad_counts.index, severidad_counts.values, color=colores)
    axes[0,0].set_title('Distribución por Severidad')
    axes[0,0].set_ylabel('Cantidad de OTs')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Agregar valores en las barras
    for bar in bars1:
        height = bar.get_height()
        axes[0,0].text(bar.get_x() + bar.get_width()/2., height,
                      f'{int(height)}', ha='center', va='bottom')
    
    # 2. Distribución por tipo de OT
    tipo_counts = ots_df['tipo'].value_counts()
    axes[0,1].pie(tipo_counts.values, labels=tipo_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0,1].set_title('Distribución por Tipo de OT')
    
    # 3. Distribución por especialidad
    especialidad_counts = ots_df['especialidad_requerida'].value_counts()
    bars3 = axes[0,2].bar(especialidad_counts.index, especialidad_counts.values, 
                         color=['lightcoral', 'lightgreen', 'lightsteelblue'])
    axes[0,2].set_title('Distribución por Especialidad')
    axes[0,2].set_ylabel('Cantidad de OTs')
    axes[0,2].tick_params(axis='x', rotation=45)
    
    for bar in bars3:
        height = bar.get_height()
        axes[0,2].text(bar.get_x() + bar.get_width()/2., height,
                      f'{int(height)}', ha='center', va='bottom')
    
    # 4. Costos por severidad
    costos_por_severidad = ots_df.groupby('severidad')['costo_total_estimado'].sum()
    colores_costos = [colores_severidad.get(sev, 'gray') for sev in costos_por_severidad.index]
    
    bars4 = axes[1,0].bar(costos_por_severidad.index, costos_por_severidad.values, color=colores_costos)
    axes[1,0].set_title('Costos Estimados por Severidad')
    axes[1,0].set_ylabel('Costo (USD)')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    for bar in bars4:
        height = bar.get_height()
        axes[1,0].text(bar.get_x() + bar.get_width()/2., height,
                      f'${height:.0f}', ha='center', va='bottom')
    
    # 5. Duración estimada por tipo
    duracion_por_tipo = ots_df.groupby('tipo')['duracion_estimada_horas'].sum()
    axes[1,1].pie(duracion_por_tipo.values, labels=duracion_por_tipo.index, 
                  autopct='%1.1f%%', startangle=90)
    axes[1,1].set_title('Duración Estimada por Tipo (horas)')
    
    # 6. Timeline de OTs por fecha programada
    ots_df['fecha_programada'] = pd.to_datetime(ots_df['fecha_programada'])
    ots_df['fecha_prog_date'] = ots_df['fecha_programada'].dt.date
    
    timeline_counts = ots_df['fecha_prog_date'].value_counts().sort_index()
    
    axes[1,2].plot(timeline_counts.index, timeline_counts.values, marker='o', linewidth=2, markersize=6)
    axes[1,2].set_title('Timeline de OTs Programadas')
    axes[1,2].set_ylabel('Cantidad de OTs')
    axes[1,2].set_xlabel('Fecha')
    axes[1,2].tick_params(axis='x', rotation=45)
    axes[1,2].grid(True, alpha=0.3)
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar gráfico
    archivo_grafico = OTS_PATH / 'analisis_ots_generadas.png'
    plt.savefig(archivo_grafico, dpi=300, bbox_inches='tight')
    print(f"📊 Gráfico guardado: {archivo_grafico}")
    plt.show()
    
    # Gráfico adicional: Matriz de prioridad vs costo
    fig2, ax2 = plt.subplots(1, 1, figsize=(10, 8))
    
    # Scatter plot prioridad vs costo
    colores_scatter = [colores_severidad.get(sev, 'gray') for sev in ots_df['severidad']]
    scatter = ax2.scatter(ots_df['prioridad'], ots_df['costo_total_estimado'], 
                         c=colores_scatter, s=100, alpha=0.7, edgecolors='black')
    
    ax2.set_xlabel('Prioridad (1=Más Alta, 4=Más Baja)')
    ax2.set_ylabel('Costo Estimado (USD)')
    ax2.set_title('Matriz Prioridad vs Costo de OTs\nSistema IA Mantenimiento Predictivo')
    ax2.grid(True, alpha=0.3)
    
    # Agregar etiquetas para OTs críticas
    for i, row in ots_df.iterrows():
        if row['severidad'] == 'CRÍTICO':
            ax2.annotate(row['numero_ot'].split('-')[-2], 
                        (row['prioridad'], row['costo_total_estimado']),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # Leyenda
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=color, label=severidad) 
                      for severidad, color in colores_severidad.items() 
                      if severidad in ots_df['severidad'].values]
    ax2.legend(handles=legend_elements, title='Severidad')
    
    plt.tight_layout()
    
    archivo_matriz = OTS_PATH / 'matriz_prioridad_costo.png'
    plt.savefig(archivo_matriz, dpi=300, bbox_inches='tight')
    print(f"📊 Matriz guardada: {archivo_matriz}")
    plt.show()

else:
    print("ℹ️ No hay OTs para visualizar")

print("\n✅ VISUALIZACIONES COMPLETADAS")

In [ ]:
# 9. 📋 RESUMEN FINAL DEL GENERADOR DE OTs
print("📋 RESUMEN FINAL DEL GENERADOR DE OTs")
print("=" * 40)

# Mostrar estadísticas finales
print(f"\n🎉 GENERACIÓN DE OTs COMPLETADA EXITOSAMENTE")
print("=" * 50)

if len(ots_generadas) > 0:
    
    # Estadísticas generales
    print(f"📊 ESTADÍSTICAS GENERALES:")
    print(f"   Total OTs generadas: {len(ots_generadas)}")
    print(f"   Anomalías procesadas: {len(anomalias_detectadas)}")
    print(f"   Tasa de conversión: {len(ots_generadas)/len(anomalias_detectadas)*100:.1f}%")
    
    # Distribución por urgencia
    criticas = len([ot for ot in ots_generadas if ot['severidad'] == 'CRÍTICO'])
    alertas = len([ot for ot in ots_generadas if ot['severidad'] == 'ALERTA'])
    atencion = len([ot for ot in ots_generadas if ot['severidad'] == 'ATENCIÓN'])
    menores = len([ot for ot in ots_generadas if ot['severidad'] == 'MENOR'])
    
    print(f"\n🚨 DISTRIBUCIÓN POR URGENCIA:")
    print(f"   🔴 Críticas: {criticas} OTs ({criticas/len(ots_generadas)*100:.1f}%)")
    print(f"   🟠 Alertas: {alertas} OTs ({alertas/len(ots_generadas)*100:.1f}%)")
    print(f"   🟡 Atención: {atencion} OTs ({atencion/len(ots_generadas)*100:.1f}%)")
    print(f"   🔵 Menores: {menores} OTs ({menores/len(ots_generadas)*100:.1f}%)")
    
    # Impacto económico
    costo_total = sum(ot['costo_total_estimado'] for ot in ots_generadas)
    costo_mano_obra = sum(ot['costo_estimado_mano_obra'] for ot in ots_generadas)
    costo_materiales = sum(ot['costo_estimado_materiales'] for ot in ots_generadas)
    
    print(f"\n💰 IMPACTO ECONÓMICO:")
    print(f"   Costo total estimado: ${costo_total:,.2f} USD")
    print(f"   Mano de obra: ${costo_mano_obra:,.2f} USD ({costo_mano_obra/costo_total*100:.1f}%)")
    print(f"   Materiales: ${costo_materiales:,.2f} USD ({costo_materiales/costo_total*100:.1f}%)")
    print(f"   Costo promedio por OT: ${costo_total/len(ots_generadas):,.2f} USD")
    
    # Recursos humanos
    duracion_total = sum(ot['duracion_estimada_horas'] for ot in ots_generadas)
    personal_total = sum(ot['personal_estimado'] for ot in ots_generadas)
    
    print(f"\n👥 RECURSOS HUMANOS:")
    print(f"   Duración total: {duracion_total:.1f} horas")
    print(f"   Personal requerido: {personal_total} personas")
    print(f"   Horas-persona total: {sum(ot['duracion_estimada_horas'] * ot['personal_estimado'] for ot in ots_generadas):.1f} h-p")
    
    # Especialidades requeridas
    especialidades_count = {}
    for ot in ots_generadas:
        esp = ot['especialidad_requerida']
        especialidades_count[esp] = especialidades_count.get(esp, 0) + 1
    
    print(f"\n🔧 ESPECIALIDADES REQUERIDAS:")
    for esp, cantidad in especialidades_count.items():
        print(f"   {esp}: {cantidad} OTs ({cantidad/len(ots_generadas)*100:.1f}%)")
    
    # Cronograma crítico
    ahora = datetime.now()
    ots_urgentes = [ot for ot in ots_generadas if ot['fecha_programada'] <= ahora + timedelta(hours=8)]
    ots_24h = [ot for ot in ots_generadas if ot['fecha_programada'] <= ahora + timedelta(hours=24)]
    
    print(f"\n⏰ CRONOGRAMA CRÍTICO:")
    print(f"   OTs urgentes (próximas 8h): {len(ots_urgentes)}")
    print(f"   OTs próximas 24h: {len(ots_24h)}")
    print(f"   OTs que requieren parada: {len([ot for ot in ots_generadas if ot['requiere_parada']])}")
    print(f"   OTs que requieren aprobación: {len([ot for ot in ots_generadas if ot['requiere_aprobacion']])}")
    
    # Archivos generados
    archivos_ots = list(OTS_PATH.glob('*'))
    archivos_ots = [f for f in archivos_ots if f.is_file()]
    
    print(f"\n📁 ARCHIVOS GENERADOS: {len(archivos_ots)}")
    for archivo in sorted(archivos_ots, key=lambda x: x.stat().st_mtime, reverse=True):
        tamaño_kb = archivo.stat().st_size / 1024
        print(f"   📄 {archivo.name} ({tamaño_kb:.1f} KB)")
    
    # Próximos pasos recomendados
    print(f"\n🚀 PRÓXIMOS PASOS RECOMENDADOS:")
    
    if criticas > 0:
        print(f"   1. 🚨 URGENTE: Revisar {criticas} OTs críticas inmediatamente")
        print(f"   2. 📞 Notificar a supervisores y especialistas eléctricos")
        print(f"   3. 🛠️ Preparar recursos para intervenciones críticas")
    
    if alertas > 0:
        print(f"   4. ⏰ Programar {alertas} OTs de alerta en próximas 8 horas")
    
    print(f"   5. 📋 Integrar OTs con sistema de gestión de mantenimiento")
    print(f"   6. 📊 Configurar monitoreo de ejecución y seguimiento")
    print(f"   7. 🔄 Validar efectividad del sistema con resultados reales")
    
    # Valor del sistema
    print(f"\n💡 VALOR DEL SISTEMA AUTOMÁTICO:")
    print(f"   ✅ Generación automática sin intervención humana")
    print(f"   ✅ Prescripciones específicas basadas en tipo de anomalía")
    print(f"   ✅ Priorización inteligente según severidad")
    print(f"   ✅ Asignación optimizada de recursos y especialidades")
    print(f"   ✅ Estimación automática de costos y tiempos")
    print(f"   ✅ Trazabilidad completa desde detección hasta ejecución")
    print(f"   ✅ Integración lista con sistemas empresariales")

else:
    print(f"ℹ️ No se generaron OTs - No se detectaron anomalías")
    print(f"✅ Sistema funcionando correctamente")
    print(f"📊 Monitoreo continuo activo")

print(f"\n" + "="*60)
print(f"🎉 GENERADOR AUTOMÁTICO DE OTs - COMPLETADO")
print(f"="*60)
print(f"📋 OTs generadas: {len(ots_generadas) if len(ots_generadas) > 0 else 0}")
print(f"💾 Archivos creados: {len(list(OTS_PATH.glob('*')))}")
print(f"📁 Ubicación: {OTS_PATH}")
print(f"⏰ Tiempo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"✅ Estado: Sistema operativo y listo para producción")
print(f"="*60)

print(f"\n🚀 ¡SISTEMA DE GENERACIÓN DE OTs LISTO PARA IMPLEMENTACIÓN!")
print(f"📖 Consulta los archivos generados en: {OTS_PATH}/")
print(f"📊 Revisa el reporte ejecutivo para detalles completos")
print(f"🔧 Las OTs están listas para integración con sistemas de mantenimiento")